In [2]:
# level91_generalized_connectedness.py
# Level-91: Generalized Spillover / Connectedness (Diebold–Yilmaz, order-invariant)
#
# Fixes included:
# - Robust yfinance loader (handles MultiIndex and ensures Series output)
# - Avoids "TypeError: 'str' object is not callable" from DataFrame.rename(s)
#
# Outputs:
# - level91_conn_full.csv
# - level91_conn_rolling_summary.csv
# - level91_conn_rolling_edges.csv
# - level91_conn_summary.json

import os
import json
import math
import argparse
from dataclasses import dataclass, asdict
from typing import Tuple, Dict, List, Optional

import numpy as np
import pandas as pd
import yfinance as yf
from statsmodels.tsa.api import VAR


# ----------------------------- Config -----------------------------
@dataclass
class Config:
    symbols: Tuple[str, ...] = ("SPY", "QQQ", "IWM", "EFA", "EEM", "TLT", "LQD", "GLD")
    start: str = "2010-01-01"

    # VAR / FEVD params
    horizon: int = 10
    maxlags: int = 5
    lags: Optional[int] = None
    ic: str = "aic"  # aic/bic/hqic/fpe

    # Rolling
    rolling: bool = True
    window: int = 756
    step: int = 5

    # Preprocessing
    use_log_returns: bool = True
    standardize: bool = True
    dropna: bool = True

    seed: int = 42

    # Output
    out_full_csv: str = "level91_conn_full.csv"
    out_roll_summary_csv: str = "level91_conn_rolling_summary.csv"
    out_roll_edges_csv: str = "level91_conn_rolling_edges.csv"
    out_json: str = "level91_conn_summary.json"


# ----------------------------- Robust yfinance loader -----------------------------
def _extract_close_series(px: pd.DataFrame, symbol: str) -> pd.Series:
    """
    Ensure we return a *Series* for the symbol's close price, regardless of whether
    yfinance returns single-index or MultiIndex columns.

    Common possibilities:
      - columns: ['Open','High','Low','Close','Adj Close','Volume']
      - columns: MultiIndex like ('Close','SPY') or ('SPY','Close')
    """
    if px is None or px.empty:
        raise RuntimeError(f"No data for {symbol}")

    # MultiIndex columns
    if isinstance(px.columns, pd.MultiIndex):
        # common layouts
        candidates = [
            ("Adj Close", symbol),
            ("Close", symbol),
            (symbol, "Adj Close"),
            (symbol, "Close"),
        ]
        for key in candidates:
            if key in px.columns:
                s = px[key].copy()
                if isinstance(s, pd.DataFrame):
                    # if duplicated columns lead to DF, take first col
                    s = s.iloc[:, 0]
                s.name = symbol
                return s

        # fallback: search any column containing symbol & close
        cols = []
        for c in px.columns:
            c0 = str(c[0]).lower()
            c1 = str(c[1]).lower()
            if (symbol.lower() in c0 or symbol.lower() in c1) and ("close" in c0 or "close" in c1):
                cols.append(c)
        if cols:
            s = px[cols[0]].copy()
            if isinstance(s, pd.DataFrame):
                s = s.iloc[:, 0]
            s.name = symbol
            return s

        raise RuntimeError(f"Could not find Close/Adj Close for {symbol} in MultiIndex columns: {px.columns}")

    # Single index columns
    for col in ["Adj Close", "Close"]:
        if col in px.columns:
            s = px[col].copy()
            if isinstance(s, pd.DataFrame):
                s = s.iloc[:, 0]
            s.name = symbol
            return s

    raise RuntimeError(f"Missing Close/Adj Close for {symbol}. Columns={list(px.columns)}")


def load_prices(symbols: Tuple[str, ...], start: str) -> pd.DataFrame:
    """
    Robust load:
    - Try single multi-ticker download (faster) and extract series
    - If any extraction fails, fallback to per-ticker download
    """
    symbols = tuple(symbols)

    # 1) try batch download
    try:
        px_all = yf.download(list(symbols), start=start, progress=False, group_by="column", auto_adjust=False)
        if px_all is not None and not px_all.empty:
            series_list = []
            for s in symbols:
                try:
                    series_list.append(_extract_close_series(px_all, s))
                except Exception:
                    series_list = []
                    break
            if series_list:
                prices = pd.concat(series_list, axis=1).sort_index()
                prices = prices.dropna(how="any")
                return prices
    except Exception:
        pass

    # 2) fallback per symbol
    frames = []
    for s in symbols:
        px = yf.download(s, start=start, progress=False, auto_adjust=False)
        close = _extract_close_series(px, s)
        frames.append(close)
    prices = pd.concat(frames, axis=1).sort_index().dropna(how="any")
    return prices


def compute_returns(prices: pd.DataFrame, use_log: bool = True) -> pd.DataFrame:
    if use_log:
        rets = np.log(prices).diff()
    else:
        rets = prices.pct_change()

    rets = rets.replace([np.inf, -np.inf], np.nan).dropna()

    # give statsmodels a freq to avoid warning + makes rolling cleaner
    rets = rets.asfreq("B")
    rets = rets.dropna()
    return rets


def zscore(df: pd.DataFrame) -> pd.DataFrame:
    mu = df.mean(axis=0)
    sd = df.std(axis=0, ddof=1).replace(0.0, np.nan)
    out = (df - mu) / sd
    return out.dropna(how="any")


# ----------------------------- VAR + Generalized FEVD -----------------------------
def select_lag(endog: pd.DataFrame, maxlags: int, ic: str) -> int:
    model = VAR(endog)
    sel = model.select_order(maxlags=maxlags)
    p = getattr(sel, ic)
    if p is None or (isinstance(p, (float, int)) and (np.isnan(p) or p <= 0)):
        return 1
    return int(p)


def fit_var(endog: pd.DataFrame, lags: Optional[int], maxlags: int, ic: str):
    model = VAR(endog)
    p = int(lags) if lags is not None else select_lag(endog, maxlags=maxlags, ic=ic)
    res = model.fit(p)
    return res, p


def generalized_fevd(res, H: int) -> np.ndarray:
    Sigma = np.asarray(res.sigma_u)
    N = Sigma.shape[0]

    Psi = np.asarray(res.ma_rep(H - 1))
    if Psi.ndim != 3 or Psi.shape[0] < H:
        raise RuntimeError(f"Unexpected ma_rep shape: {Psi.shape}")

    denom = np.zeros(N, dtype=float)
    for i in range(N):
        ei = np.zeros(N); ei[i] = 1.0
        s = 0.0
        for h in range(H):
            Ph = Psi[h]
            s += float(ei @ Ph @ Sigma @ Ph.T @ ei)
        denom[i] = max(s, 1e-18)

    Theta = np.zeros((N, N), dtype=float)
    diag = np.diag(Sigma).copy()
    diag = np.where(diag <= 0, np.nan, diag)

    for i in range(N):
        ei = np.zeros(N); ei[i] = 1.0
        for j in range(N):
            ej = np.zeros(N); ej[j] = 1.0
            numer = 0.0
            for h in range(H):
                Ph = Psi[h]
                s_ijh = float(ei @ Ph @ Sigma @ ej)
                numer += s_ijh * s_ijh
            Theta[i, j] = 0.0 if np.isnan(diag[j]) else numer / (diag[j] * denom[i])

    row_sums = Theta.sum(axis=1, keepdims=True)
    row_sums = np.where(row_sums <= 0, 1.0, row_sums)
    Theta = Theta / row_sums
    return Theta


def connectedness_from_theta(theta: np.ndarray, labels: List[str]) -> Dict[str, object]:
    N = theta.shape[0]
    off = theta.copy()
    np.fill_diagonal(off, 0.0)

    FROM = off.sum(axis=1)
    TO = off.sum(axis=0)
    NET = TO - FROM
    TCI = 100.0 * off.sum() / N

    table = pd.DataFrame(theta * 100.0, index=labels, columns=labels)
    table["FROM"] = FROM * 100.0
    table.loc["TO", :] = list(TO * 100.0) + [np.nan]
    table.loc["NET", :] = list(NET * 100.0) + [np.nan]
    table.loc["TCI", :] = [np.nan] * (N + 1)
    table.loc["TCI", labels[0]] = TCI

    stats = {
        "TCI": float(TCI),
        "FROM": {labels[i]: float(FROM[i] * 100.0) for i in range(N)},
        "TO": {labels[i]: float(TO[i] * 100.0) for i in range(N)},
        "NET": {labels[i]: float(NET[i] * 100.0) for i in range(N)},
    }
    return {"table": table, "stats": stats}


def theta_to_edges(theta: np.ndarray, labels: List[str]) -> pd.DataFrame:
    rows = []
    N = theta.shape[0]
    for i, src in enumerate(labels):
        for j, dst in enumerate(labels):
            if i == j:
                continue
            rows.append({"src": src, "dst": dst, "weight_pct": float(theta[j, i] * 100.0)})
    return pd.DataFrame(rows)


def compute_connectedness(endog: pd.DataFrame, cfg: Config) -> Dict[str, object]:
    res, p = fit_var(endog, cfg.lags, cfg.maxlags, cfg.ic)
    theta = generalized_fevd(res, cfg.horizon)
    labels = list(endog.columns)
    conn = connectedness_from_theta(theta, labels)
    edges = theta_to_edges(theta, labels)
    return {"lags_used": p, "theta": theta, "table": conn["table"], "stats": conn["stats"], "edges": edges}


# ----------------------------- Full + Rolling -----------------------------
def run_full_sample(rets: pd.DataFrame, cfg: Config) -> Dict[str, object]:
    data = zscore(rets) if cfg.standardize else rets.copy()
    return compute_connectedness(data, cfg)


def run_rolling(rets: pd.DataFrame, cfg: Config) -> Dict[str, object]:
    idx = rets.index
    labels = list(rets.columns)

    roll_rows = []
    edge_rows = []

    ends = list(range(cfg.window, len(rets), cfg.step))
    for k, end in enumerate(ends, start=1):
        w = rets.iloc[end - cfg.window:end].copy()
        if cfg.standardize:
            w = zscore(w)
        if len(w) < max(50, cfg.window // 2):
            continue

        try:
            out = compute_connectedness(w, cfg)
        except Exception:
            continue

        dt = idx[end - 1]
        stats = out["stats"]

        row = {"date": dt, "TCI": stats["TCI"], "lags_used": out["lags_used"]}
        for a in labels:
            row[f"TO_{a}"] = stats["TO"][a]
            row[f"FROM_{a}"] = stats["FROM"][a]
            row[f"NET_{a}"] = stats["NET"][a]
        roll_rows.append(row)

        ed = out["edges"].copy()
        ed["date"] = dt
        edge_rows.append(ed)

        if k % max(1, (len(ends) // 10)) == 0:
            print(f"[INFO] Rolling progress: {k}/{len(ends)} windows...")

    roll_df = pd.DataFrame(roll_rows).set_index("date").sort_index()
    edges_df = pd.concat(edge_rows, axis=0, ignore_index=True) if edge_rows else pd.DataFrame(
        columns=["src", "dst", "weight_pct", "date"]
    )
    return {"rolling_summary": roll_df, "rolling_edges": edges_df}


def save_outputs(full_out: Dict[str, object], roll_out: Optional[Dict[str, object]], cfg: Config, data_window: Dict[str, object]) -> None:
    os.makedirs(os.path.dirname(cfg.out_full_csv) or ".", exist_ok=True)
    os.makedirs(os.path.dirname(cfg.out_roll_summary_csv) or ".", exist_ok=True)
    os.makedirs(os.path.dirname(cfg.out_roll_edges_csv) or ".", exist_ok=True)
    os.makedirs(os.path.dirname(cfg.out_json) or ".", exist_ok=True)

    full_out["table"].to_csv(cfg.out_full_csv)

    if roll_out is not None:
        roll_out["rolling_summary"].to_csv(cfg.out_roll_summary_csv)
        roll_out["rolling_edges"].to_csv(cfg.out_roll_edges_csv, index=False)

    summary = {
        "config": asdict(cfg),
        "data_window": data_window,
        "full_sample": {
            "lags_used": int(full_out["lags_used"]),
            "TCI": float(full_out["stats"]["TCI"]),
            "NET_top_positive": sorted(full_out["stats"]["NET"].items(), key=lambda kv: kv[1], reverse=True)[:5],
            "NET_top_negative": sorted(full_out["stats"]["NET"].items(), key=lambda kv: kv[1])[:5],
        }
    }
    if roll_out is not None and not roll_out["rolling_summary"].empty:
        roll_df = roll_out["rolling_summary"]
        summary["rolling"] = {
            "n_points": int(len(roll_df)),
            "tci_min": float(roll_df["TCI"].min()),
            "tci_max": float(roll_df["TCI"].max()),
            "tci_last": float(roll_df["TCI"].iloc[-1]),
            "date_first": str(roll_df.index.min().date()),
            "date_last": str(roll_df.index.max().date()),
        }

    with open(cfg.out_json, "w", encoding="utf-8") as f:
        json.dump(summary, f, indent=2)

    print(f"[OK] Saved full connectedness table → {cfg.out_full_csv}")
    if roll_out is not None:
        print(f"[OK] Saved rolling summary → {cfg.out_roll_summary_csv}")
        print(f"[OK] Saved rolling edges → {cfg.out_roll_edges_csv}")
    print(f"[OK] Saved summary → {cfg.out_json}")


def run_pipeline(cfg: Config) -> None:
    np.random.seed(cfg.seed)

    print(f"[INFO] Downloading prices for {cfg.symbols} from {cfg.start} ...")
    prices = load_prices(cfg.symbols, cfg.start)
    rets = compute_returns(prices, use_log=cfg.use_log_returns)
    if cfg.dropna:
        rets = rets.dropna(how="any")

    # enforce symbol order
    rets = rets.loc[:, list(cfg.symbols)]

    data_window = {
        "start": str(rets.index.min().date()),
        "end": str(rets.index.max().date()),
        "n_returns": int(len(rets)),
        "assets": int(rets.shape[1]),
    }

    print(f"[INFO] Got {len(prices)} price rows, {len(rets)} return rows, assets={rets.shape[1]}")
    print(f"[INFO] Full-sample VAR+GFEVD: horizon={cfg.horizon}, lags={cfg.lags or 'auto('+cfg.ic+')'}")

    full_out = run_full_sample(rets, cfg)
    print(f"[OK] Full-sample TCI={full_out['stats']['TCI']:.2f} | lags_used={full_out['lags_used']}")

    roll_out = None
    if cfg.rolling:
        if len(rets) < cfg.window + 50:
            print("[WARN] Not enough data for rolling; skipping rolling outputs.")
        else:
            print(f"[INFO] Rolling connectedness: window={cfg.window}, step={cfg.step}")
            roll_out = run_rolling(rets, cfg)
            if not roll_out["rolling_summary"].empty:
                print(f"[OK] Rolling points={len(roll_out['rolling_summary'])} | last TCI={roll_out['rolling_summary']['TCI'].iloc[-1]:.2f}")

    save_outputs(full_out, roll_out, cfg, data_window)

    net = full_out["stats"]["NET"]
    net_sorted = sorted(net.items(), key=lambda kv: kv[1], reverse=True)
    print("[TOP] NET transmitters:")
    for k, v in net_sorted[:min(5, len(net_sorted))]:
        print(f"  {k:>5s}  NET={v:.2f}")
    print("[TOP] NET receivers:")
    for k, v in net_sorted[::-1][:min(5, len(net_sorted))]:
        print(f"  {k:>5s}  NET={v:.2f}")


# ----------------------------- CLI -----------------------------
def parse_args() -> Config:
    p = argparse.ArgumentParser(description="Level-91: Generalized (order-invariant) connectedness via VAR + GFEVD")

    p.add_argument("--start", type=str, default=Config.start)
    p.add_argument("--symbols", nargs="+", default=list(Config.symbols))

    p.add_argument("--horizon", type=int, default=Config.horizon)
    p.add_argument("--maxlags", type=int, default=Config.maxlags)
    p.add_argument("--lags", type=int, default=None)
    p.add_argument("--ic", type=str, default=Config.ic, choices=["aic", "bic", "hqic", "fpe"])

    p.add_argument("--no-rolling", action="store_true")
    p.add_argument("--window", type=int, default=Config.window)
    p.add_argument("--step", type=int, default=Config.step)

    p.add_argument("--no-standardize", action="store_true")
    p.add_argument("--simple-returns", action="store_true")

    p.add_argument("--seed", type=int, default=Config.seed)

    p.add_argument("--full-csv", type=str, default=Config.out_full_csv)
    p.add_argument("--roll-csv", type=str, default=Config.out_roll_summary_csv)
    p.add_argument("--edges-csv", type=str, default=Config.out_roll_edges_csv)
    p.add_argument("--json", type=str, default=Config.out_json)

    a = p.parse_args()
    return Config(
        symbols=tuple(a.symbols),
        start=a.start,
        horizon=int(a.horizon),
        maxlags=int(a.maxlags),
        lags=(int(a.lags) if a.lags is not None else None),
        ic=str(a.ic),
        rolling=(not a.no_rolling),
        window=int(a.window),
        step=int(a.step),
        use_log_returns=(not a.simple_returns),
        standardize=(not a.no_standardize),
        seed=int(a.seed),
        out_full_csv=a.full_csv,
        out_roll_summary_csv=a.roll_csv,
        out_roll_edges_csv=a.edges_csv,
        out_json=a.json,
    )


def main() -> None:
    cfg = parse_args()
    run_pipeline(cfg)


if __name__ == "__main__":
    # Jupyter/PyCharm shim: strip "-f kernel.json" etc.
    import sys
    sys.argv = [sys.argv[0]] + [
        arg for arg in sys.argv[1:]
        if arg != "-f" and not (arg.endswith(".json") and "kernel" in arg)
    ]
    main()


[INFO] Downloading prices for ('SPY', 'QQQ', 'IWM', 'EFA', 'EEM', 'TLT', 'LQD', 'GLD') from 2010-01-01 ...
[INFO] Got 4021 price rows, 4020 return rows, assets=8
[INFO] Full-sample VAR+GFEVD: horizon=10, lags=auto(aic)


C:\Users\adity\PyCharmMiscProject\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\adity\PyCharmMiscProject\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


[OK] Full-sample TCI=59.42 | lags_used=4
[INFO] Rolling connectedness: window=756, step=5


C:\Users\adity\PyCharmMiscProject\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\adity\PyCharmMiscProject\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\adity\PyCharmMiscProject\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\adity\PyCharmMiscProject\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so

[INFO] Rolling progress: 65/653 windows...


...

[INFO] Rolling progress: 585/653 windows...


C:\Users\adity\PyCharmMiscProject\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\adity\PyCharmMiscProject\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\adity\PyCharmMiscProject\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\adity\PyCharmMiscProject\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so

[INFO] Rolling progress: 650/653 windows...
[OK] Rolling points=653 | last TCI=60.36


C:\Users\adity\PyCharmMiscProject\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\adity\PyCharmMiscProject\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\adity\PyCharmMiscProject\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\adity\PyCharmMiscProject\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so

[OK] Saved full connectedness table → level91_conn_full.csv
[OK] Saved rolling summary → level91_conn_rolling_summary.csv
[OK] Saved rolling edges → level91_conn_rolling_edges.csv
[OK] Saved summary → level91_conn_summary.json
[TOP] NET transmitters:
    SPY  NET=13.47
    EFA  NET=6.40
    IWM  NET=1.16
    QQQ  NET=1.11
    EEM  NET=-1.39
[TOP] NET receivers:
    GLD  NET=-7.61
    TLT  NET=-6.66
    LQD  NET=-6.48
    EEM  NET=-1.39
    QQQ  NET=1.11
